In [39]:
# Q.1
import urllib.request

def download_file(url,filename=None):
    urllib.request.urlretrieve(url,filename)

download_file('https://assets-datascientest.s3.eu-west-1.amazonaws.com/gps_app.csv','gps_app.csv')

In [89]:
# Q.4.1

In [90]:
download_file('https://assets-datascientest.s3.eu-west-1.amazonaws.com/creditcard.csv','creditcard.csv')

In [42]:
# Q.2

In [40]:
from pyspark.sql import SparkSession

spark = SparkSession\
        .builder\
        .appName("Cours de Spark")\
        .master("local[*]")\
        .getOrCreate()
sc = spark.sparkContext

In [41]:
raw_app = spark.read.option("header", True)\
                    .option("inferSchema", True)\
                    .option("escape", "\"")\
                    .csv("gps_app.csv")

In [43]:
raw_app.printSchema()

root
 |-- App: string (nullable = true)
 |-- Category: string (nullable = true)
 |-- Rating: double (nullable = true)
 |-- Reviews: string (nullable = true)
 |-- Size: string (nullable = true)
 |-- Installs: string (nullable = true)
 |-- Type: string (nullable = true)
 |-- Price: string (nullable = true)
 |-- Content Rating: string (nullable = true)
 |-- Genres: string (nullable = true)
 |-- Last Updated: string (nullable = true)
 |-- Current Ver: string (nullable = true)
 |-- Android Ver: string (nullable = true)



In [44]:
raw_app.columns

['App',
 'Category',
 'Rating',
 'Reviews',
 'Size',
 'Installs',
 'Type',
 'Price',
 'Content Rating',
 'Genres',
 'Last Updated',
 'Current Ver',
 'Android Ver']

In [45]:
import re
p = re.compile(' ')

df_renamed = raw_app
for c in raw_app.columns:
    c2=p.sub('_', c).lower()
    print(c,'->',c2)
    df_renamed = df_renamed.withColumnRenamed(c, c2)
df_renamed.columns

App -> app
Category -> category
Rating -> rating
Reviews -> reviews
Size -> size
Installs -> installs
Type -> type
Price -> price
Content Rating -> content_rating
Genres -> genres
Last Updated -> last_updated
Current Ver -> current_ver
Android Ver -> android_ver


['app',
 'category',
 'rating',
 'reviews',
 'size',
 'installs',
 'type',
 'price',
 'content_rating',
 'genres',
 'last_updated',
 'current_ver',
 'android_ver']

In [46]:
raw_app = df_renamed

In [47]:
# Q.3.1

In [48]:
raw_app.summary().show()

[Stage 43:>                                                         (0 + 1) / 1]

+-------+--------------------+--------+------+------------------+------------------+--------+-----+--------+---------------+------+-----------------+-------------+------------------+
|summary|                 app|category|rating|           reviews|              size|installs| type|   price| content_rating|genres|     last_updated|  current_ver|       android_ver|
+-------+--------------------+--------+------+------------------+------------------+--------+-----+--------+---------------+------+-----------------+-------------+------------------+
|  count|               10841|   10841| 10841|             10841|             10841|   10841|10841|   10841|          10840| 10841|            10841|        10840|             10840|
|   mean|                NULL|     1.9|   NaN|444152.89603321033|              NULL|     0.0|  NaN|     0.0|           NULL|  NULL|             NULL|          NaN|               NaN|
| stddev|                NULL|    NULL|   NaN|2927760.6038856646|              NULL| 

In [49]:
# Je choisis la médiane car en principe, la moyenne peut être moins représentative d'une "valeur typique",
# notamment car elle peut être sujette à des déséquilibres.

from pyspark.ml.feature import Imputer

impute_median = Imputer(strategy="median",\
                        inputCol="rating",\
                        outputCol="rating")

model_median = impute_median.fit(raw_app)

raw_app = model_median.transform(raw_app)

raw_app.summary().show()

[Stage 49:>                                                         (0 + 1) / 1]

+-------+--------------------+--------+------------------+------------------+------------------+--------+-----+--------+---------------+------+-----------------+-------------+------------------+
|summary|                 app|category|            rating|           reviews|              size|installs| type|   price| content_rating|genres|     last_updated|  current_ver|       android_ver|
+-------+--------------------+--------+------------------+------------------+------------------+--------+-----+--------+---------------+------+-----------------+-------------+------------------+
|  count|               10841|   10841|             10841|             10841|             10841|   10841|10841|   10841|          10840| 10841|            10841|        10840|             10840|
|   mean|                NULL|     1.9| 4.207840605110369|444152.89603321033|              NULL|     0.0|  NaN|     0.0|           NULL|  NULL|             NULL|          NaN|               NaN|
| stddev|                

In [50]:
# Q.3.2
raw_app.select('type').distinct().show()

+----+
|type|
+----+
|   0|
| NaN|
|Free|
|Paid|
+----+



In [51]:
raw_app.groupBy('type').count().show()


+----+-----+
|type|count|
+----+-----+
|   0|    1|
| NaN|    1|
|Free|10039|
|Paid|  800|
+----+-----+



In [52]:
# La valeur le plus logique est le mode, donc "Free".

In [53]:
from pyspark.sql.functions import *

mode="Free"
df = raw_app.withColumn("type", when(isnan("type") | isnull("type"), mode)\
                                              .otherwise(col("type")))


In [54]:
raw_app.filter(isnan("type")).count()

1

In [55]:
raw_app.filter(isnull("type")).count()

0

In [56]:
df.filter(isnan("type")).count()

0

In [57]:
df.filter(isnull("type")).count()

0

In [58]:
raw_app=df

In [59]:
raw_app.groupBy('type').count().show()


+----+-----+
|type|count|
+----+-----+
|   0|    1|
|Free|10040|
|Paid|  800|
+----+-----+



In [60]:
# Q.3.3
raw_app.select('type').distinct().show()
# La valeur 0 est invalide.

+----+
|type|
+----+
|   0|
|Free|
|Paid|
+----+



In [62]:
raw_app.filter(col('type')=="0").show()

+--------------------+--------+------+-------+------+--------+----+--------+--------------+-----------------+------------+-----------+-----------+
|                 app|category|rating|reviews|  size|installs|type|   price|content_rating|           genres|last_updated|current_ver|android_ver|
+--------------------+--------+------+-------+------+--------+----+--------+--------------+-----------------+------------+-----------+-----------+
|Life Made WI-Fi T...|     1.9|  19.0|   3.0M|1,000+|    Free|   0|Everyone|          NULL|February 11, 2018|      1.0.19| 4.0 and up|       NULL|
+--------------------+--------+------+-------+------+--------+----+--------+--------------+-----------------+------------+-----------+-----------+



In [66]:
raw_app=raw_app.filter(col('type')!="0")

In [ ]:
#Q.3.4

In [68]:
raw_app.filter(isnan(col('current_ver')) | isnull(col('current_ver'))).groupBy('current_ver').count().show()


+-----------+-----+
|current_ver|count|
+-----------+-----+
|       NULL|    1|
|        NaN|    7|
+-----------+-----+



In [73]:
mode=raw_app.groupBy('current_ver').count().orderBy('count', ascending=False).head()['current_ver']
mode

'Varies with device'

In [75]:
raw_app = raw_app.withColumn("current_ver", when(isnan("current_ver") | isnull("current_ver"), mode)\
                                              .otherwise(col("current_ver")))


In [78]:
mode=raw_app.groupBy('android_ver').count().orderBy('count', ascending=False).head()['android_ver']
mode

'4.1 and up'

In [80]:
raw_app = raw_app.withColumn("android_ver", when(isnan("android_ver") | isnull("android_ver"), mode)\
                                              .otherwise(col("android_ver")))


In [ ]:
# Q.3.5

In [86]:
#import numpy as np
#from pyspark.sql.functions import isnan, isnull
from pyspark.sql.types import BooleanType

def getMissingValues(dataframe):
  count = dataframe.count()
  columns = dataframe.columns
  nan_count = []
  # we can't check for nan in a boolean type column
  for column in columns:
    if dataframe.schema[column].dataType == BooleanType():
      nan_count.append(0)
    else:
      nan_count.append(dataframe.where(isnan(col(column))).count())
  null_count = [dataframe.where(isnull(col(column))).count() for column in columns]
  return([count, columns, nan_count, null_count])

def missingTable(stats):
  count, columns, nan_count, null_count = stats
  count = str(count)
  nan_count = [str(element) for element in nan_count]
  null_count = [str(element) for element in null_count]
  max_init = np.max([len(str(count)), 10])
  line1 = "+" + max_init*"-" + "+"
  line2 = "|" + (max_init-len(count))*" " + count + "|"
  line3 = "|" + (max_init-9)*" " + "nan count|"
  line4 = "|" + (max_init-10)*" " + "null count|"
  for i in range(len(columns)):
    max_column = np.max([len(columns[i]),\
                        len(nan_count[i]),\
                        len(null_count[i])])
    line1 += max_column*"-" + "+"
    line2 += (max_column - len(columns[i]))*" " + columns[i] + "|"
    line3 += (max_column - len(nan_count[i]))*" " + nan_count[i] + "|"
    line4 += (max_column - len(null_count[i]))*" " + null_count[i] + "|"
  lines = f"{line1}\n{line2}\n{line1}\n{line3}\n{line4}\n{line1}"
  print(lines)


In [87]:
missingTable(getMissingValues(raw_app))


+----------+---+--------+------+-------+----+--------+----+-----+--------------+------+------------+-----------+-----------+
|     10840|app|category|rating|reviews|size|installs|type|price|content_rating|genres|last_updated|current_ver|android_ver|
+----------+---+--------+------+-------+----+--------+----+-----+--------------+------+------------+-----------+-----------+
| nan count|  0|       0|     0|      0|   0|       0|   0|    0|             0|     0|           0|          0|          0|
|null count|  0|       0|     0|      0|   0|       0|   0|    0|             0|     0|           0|          0|          0|
+----------+---+--------+------+-------+----+--------+----+-----+--------------+------+------------+-----------+-----------+


In [ ]:
#Q.4.2

In [91]:
df = spark.read.option("header", True)\
               .option("inferSchema", True)\
               .option("escape", "\"")\
               .csv("creditcard.csv")

In [92]:
df.printSchema()

root
 |-- Time: double (nullable = true)
 |-- V1: double (nullable = true)
 |-- V2: double (nullable = true)
 |-- V3: double (nullable = true)
 |-- V4: double (nullable = true)
 |-- V5: double (nullable = true)
 |-- V6: double (nullable = true)
 |-- V7: double (nullable = true)
 |-- V8: double (nullable = true)
 |-- V9: double (nullable = true)
 |-- V10: double (nullable = true)
 |-- V11: double (nullable = true)
 |-- V12: double (nullable = true)
 |-- V13: double (nullable = true)
 |-- V14: double (nullable = true)
 |-- V15: double (nullable = true)
 |-- V16: double (nullable = true)
 |-- V17: double (nullable = true)
 |-- V18: double (nullable = true)
 |-- V19: double (nullable = true)
 |-- V20: double (nullable = true)
 |-- V21: double (nullable = true)
 |-- V22: double (nullable = true)
 |-- V23: double (nullable = true)
 |-- V24: double (nullable = true)
 |-- V25: double (nullable = true)
 |-- V26: double (nullable = true)
 |-- V27: double (nullable = true)
 |-- V28: double (nulla

In [94]:
df.summary().show()

[Stage 202:>                                                        (0 + 1) / 1]

+-------+------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+------------------+--------------------+
|summary|              Time|                  V1|                  V2|                  V3|                  V4|                  V5|                  V6|                  V7|                  V8|                  V9|                 V10|                 V11|                 V12|                 V13|                 V14|                 V15|

In [95]:
missingTable(getMissingValues(df))

[Stage 389:>                                                        (0 + 2) / 2]

+----------+----+--+--+--+--+--+--+--+--+--+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+------+-----+
|    284807|Time|V1|V2|V3|V4|V5|V6|V7|V8|V9|V10|V11|V12|V13|V14|V15|V16|V17|V18|V19|V20|V21|V22|V23|V24|V25|V26|V27|V28|Amount|Class|
+----------+----+--+--+--+--+--+--+--+--+--+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+------+-----+
| nan count|   0| 0| 0| 0| 0| 0| 0| 0| 0| 0|  0|  0|  0|  0|  0|  0|  0|  0|  0|  0|  0|  0|  0|  0|  0|  0|  0|  0|  0|     0|    0|
|null count|   0| 0| 0| 0| 0| 0| 0| 0| 0| 0|  0|  0|  0|  0|  0|  0|  0|  0|  0|  0|  0|  0|  0|  0|  0|  0|  0|  0|  0|     0|    0|
+----------+----+--+--+--+--+--+--+--+--+--+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+------+-----+


In [110]:
df = df.withColumnRenamed('Amount', 'label')


In [111]:
train, test = df.randomSplit([0.9, 0.1], seed=12345)

In [112]:
cols=list(set(train.columns)-{'label'})
cols

['V7',
 'V25',
 'V4',
 'V8',
 'V5',
 'V6',
 'V24',
 'V23',
 'V1',
 'V18',
 'V14',
 'V22',
 'V10',
 'V9',
 'V20',
 'V12',
 'V27',
 'V28',
 'V26',
 'V16',
 'V3',
 'V13',
 'V11',
 'V15',
 'Time',
 'V2',
 'V21',
 'V17',
 'Class',
 'V19']

In [105]:
from pyspark.ml.feature import StandardScaler, VectorAssembler

In [113]:
va = VectorAssembler(inputCols=cols, outputCol="numerical_features")

scaler = StandardScaler(inputCol="numerical_features", outputCol="features")

In [ ]:
from pyspark.ml import Pipeline
from pyspark.ml.regression import LinearRegression
from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.ml.tuning import CrossValidator, ParamGridBuilder

lr = LinearRegression()

pipeline = Pipeline(stages=[va, scaler, lr])

evaluator = RegressionEvaluator()

output = ParamGridBuilder() \
    .baseOn({lr.labelCol: 'label'}) \
    .baseOn([lr.predictionCol, 'prediction']) \
    .addGrid(lr.regParam, [0.3, 1.3]) \
    .addGrid(lr.elasticNetParam, [0.6, 0.8]) \
    .addGrid(lr.maxIter, [5, 10]) \
    .build()

cv = CrossValidator(estimator=pipeline,\
                    estimatorParamMaps=output,\
                    evaluator=evaluator,\
                    numFolds=3)

cvModel = cv.fit(train)

test_predict = cvModel.transform(test)

In [117]:
print(evaluator.getMetricName(), evaluator.evaluate(test_predict))


[Stage 578:>                                                        (0 + 2) / 2]

rmse 78.7331229213159
